In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import cv2
import numpy as np
from tensorflow import keras

# data
train_path = 'drive/MyDrive/preprocessed_data_for_cnn/train'
validation_path = 'drive/MyDrive/preprocessed_data_for_cnn/validation'
test_path = 'drive/MyDrive/preprocessed_data_for_cnn/test'

# 5 facial expressions
emotion_list = ['joy', 'embarrassed', 'anger', 'sad', 'neutral']

In [3]:
train_images = []
train_answers = []

for emotion in emotion_list:
    path = os.path.join(train_path, emotion)
    cnt = 0
    for image_name in os.listdir(path):
        image_path = os.path.join(path, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        train_images.append(image)
        train_answers.append(emotion_list.index(emotion))
        cnt += 1
        if cnt == 4000: break

train_images = np.array(train_images)
train_answers = np.array(train_answers)
train_index = np.arange(train_answers.shape[0])
np.random.shuffle(train_index)
train_images = train_images[train_index]
train_answers = train_answers[train_index]
train_scaled = train_images.reshape(-1, 128, 128, 1) / 255.0
print(len(train_scaled))

20000


In [4]:
validation_images = []
validation_answers = []

for emotion in emotion_list:
    path = os.path.join(validation_path, emotion)
    cnt = 0
    for image_name in os.listdir(path):
        image_path = os.path.join(path, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        validation_images.append(image)
        validation_answers.append(emotion_list.index(emotion))
        cnt += 1
        if cnt == 800: break

validation_images = np.array(validation_images)
validation_answers = np.array(validation_answers)
validation_index = np.arange(validation_answers.shape[0])
np.random.shuffle(validation_index)
validation_images = validation_images[validation_index]
validation_answers = validation_answers[validation_index]
validation_scaled = validation_images.reshape(-1, 128, 128, 1) / 255.0
print(len(validation_scaled))

4000


In [5]:
test_images = []
test_answers = []

for emotion in emotion_list:
    path = os.path.join(test_path, emotion)
    cnt = 0
    for image_name in os.listdir(path):
        image_path = os.path.join(path, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        test_images.append(image)
        test_answers.append(emotion_list.index(emotion))
        cnt += 1
        if cnt == 600: break

test_images = np.array(test_images)
test_answers = np.array(test_answers)
test_scaled = test_images.reshape(-1, 128, 128, 1) / 255.0
print(len(test_scaled))

3000


In [6]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(
    32,
    kernel_size=3,
    activation='relu',
    padding='same',
    input_shape=(128, 128, 1)
))
model.add(keras.layers.MaxPooling2D(2))

model.add(keras.layers.Conv2D(
    64,
    kernel_size=3,
    activation='relu',
    padding='same'
))
model.add(keras.layers.MaxPooling2D(2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(5, activation='softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 65536)             0         
                                                                 
 dense (Dense)               (None, 256)               1

In [7]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics='accuracy'
)
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
history = model.fit(
    train_scaled,
    train_answers,
    epochs=20,
    validation_data=(validation_scaled, validation_answers),
    callbacks=[checkpoint_cb, early_stopping_cb]
)

Epoch 1/20
625/625 [==============================] - 17s 23ms/step - loss: 1.4882 - accuracy: 0.3133 - val_loss: 1.1460 - val_accuracy: 0.5228
Epoch 2/20
625/625 [==============================] - 13s 20ms/step - loss: 0.9882 - accuracy: 0.6092 - val_loss: 0.9348 - val_accuracy: 0.6323
Epoch 3/20
625/625 [==============================] - 13s 20ms/step - loss: 0.7678 - accuracy: 0.7075 - val_loss: 0.8195 - val_accuracy: 0.6840
Epoch 4/20
625/625 [==============================] - 13s 20ms/step - loss: 0.6274 - accuracy: 0.7624 - val_loss: 0.7768 - val_accuracy: 0.7268
Epoch 5/20
625/625 [==============================] - 13s 20ms/step - loss: 0.5255 - accuracy: 0.8045 - val_loss: 0.7443 - val_accuracy: 0.7390
Epoch 6/20
625/625 [==============================] - 13s 21ms/step - loss: 0.4414 - accuracy: 0.8380 - val_loss: 0.7915 - val_accuracy: 0.7303
Epoch 7/20
625/625 [==============================] - 13s 21ms/step - loss: 0.3633 - accuracy: 0.8679 - val_loss: 0.8391 - val_accuracy:

In [8]:
model.evaluate(test_scaled, test_answers)

94/94 [==============================] - 1s 10ms/step - loss: 0.9284 - accuracy: 0.6870


[0.9283934831619263, 0.6869999766349792]